In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
import handy
import sys, os
import sigpy as sp
import scipy
from joblib import Parallel, delayed
import time

In [2]:
shelp = loadmat('sref.mat')
csm = shelp['s_ref']
data = handy.read_matlab('raw_data.mat')
ksp = data['raw']
raw = ksp['real']+1j*ksp['imag']


In [48]:
img = (sp.ifft(raw, axes = (2,3)))
img.shape
img = permute(permute(img,[3,1]),[3,0])
img.shape

1 2
2 3
0 1
1 2
2 3


(96, 96, 1, 16)

In [49]:
#handy.plot_patterns(abs(img),4,'jet')

In [50]:
#handy.plot_patterns(abs(handy.swappy(csm)).squeeze(),4,'jet')

In [51]:
def adaptive_sense(kspace, w = 5):
    """Approximate CSM maps; W. Marcellin, Adaptive Reconstruction of Phased Array MR Imagery
    Parameters:
        kspace (np.array): matrix containing the data
        w (int): size of kernel
    Returns:
        CSM (np.array): numpy array of size Nx, Ny, Nz, Nc containing the CSM maps
    """
    img = np.copy(kspace)
    Nx,Ny,Nz,Nc = img.shape
    S = np.zeros((Nx,Ny,Nz,Nc),dtype=complex)
    M = np.zeros((Nx,Ny,Nz),dtype=complex)
    for i in np.arange(Nx):
        ii = np.arange(np.max((i-w,0)),np.min((i+w+1,Nx)))
        for j in np.arange(Ny):
            jj = np.arange(np.max((j-w,0)),np.min((j+w+1,Ny)))
            for k in np.arange(Nz):
                kk = np.arange(np.max((k-w,0)),np.min((k+w+1,Nz)))
                coord = np.ix_(ii,jj,kk)
                kernel = np.reshape(img[coord],(-1,Nc))
                e, vr = np.linalg.eig(np.conj(kernel.conj().T@kernel))
                largest = np.argmax(e)
                D = e[largest]
                V = vr[:,largest]
                S[i,j,k,:] = V*np.exp(-1j*np.angle(V[0]))
                M[i,j,k] = np.sqrt(D)
    CSM=S*(M>(0.1*np.max(M)))[:,:,:,None]
    return CSM

In [52]:
dis = np.repeat(img,20,axis=2)
start = time.time()
#data = adaptive_sense(dis)
print(time.time()-start)

0.0


In [53]:
handy.plot_patterns(handy.swappy(abs(data.squeeze()[:,:,10])),4,'jet')

AttributeError: 'dict' object has no attribute 'squeeze'

In [54]:
def func():
    m=np.random.rand(6,6)*10
    s=np.zeros((100*100*100,6,6))
    for i in range(10):
        for j in range(10):
            for k in range(10):
                s[i] = scipy.linalg.expm(m)
    return s


In [ ]:

Nx,Ny,Nz,Nc = img.shape
S = np.zeros((Nx,Ny,Nz,Nc),dtype=complex)
M = np.zeros((Nx,Ny,Nz),dtype=complex)
for i in np.arange(Nx):
    ii = np.arange(np.max((i-w,0)),np.min((i+w+1,Nx)))
    for j in np.arange(Ny):
        jj = np.arange(np.max((j-w,0)),np.min((j+w+1,Ny)))
        for k in np.arange(Nz):
            kk = np.arange(np.max((k-w,0)),np.min((k+w+1,Nz)))
            coord = np.ix_(ii,jj,kk)
            kernel = np.reshape(img[coord],(-1,Nc))
            e, vr = np.linalg.eig(np.conj(kernel.conj().T@kernel))
            
            largest = np.argmax(e)
            D = e[largest]
            V = vr[:,largest]
            S[i,j,k,:] = V*np.exp(-1j*np.angle(V[0]))
            M[i,j,k] = np.sqrt(D)
CSM=S*(M>(0.1*np.max(M)))[:,:,:,None]

In [ ]:
%timeit adaptive_sense(img,w=5)

In [56]:
def func(img,i,j,k,w=5):
    ii = np.arange(np.max((i-w,0)),np.min((i+w+1,Nx)))
    jj = np.arange(np.max((j-w,0)),np.min((j+w+1,Ny)))
    kk = np.arange(np.max((k-w,0)),np.min((k+w+1,Nz)))
    coord = np.ix_(ii,jj,kk)
    kernel = np.reshape(img[coord],(-1,Nc))
    e, vr = np.linalg.eig(np.conj(kernel.conj().T@kernel))
    largest = np.argmax(e)
    D = e[largest]
    V = vr[:,largest]
    return (V*np.exp(-1j*np.angle(V[0])), np.sqrt(D),[i,j,k])

In [57]:
start = time.time()
Nx,Ny,Nz,Nc = dis.shape
csm_pll = Parallel(n_jobs=-1)(delayed(func)(dis,i,j,k,5) for i in np.arange(Nx) for j in np.arange(Ny) for k in np.arange(Nz))
print(time.time()-start)

43.4930739402771


In [65]:
S = np.zeros((Nx,Ny,Nz,Nc),dtype=complex)
M = np.zeros((Nx,Ny,Nz),dtype=complex)
for i in csm_pll:
    c = i[2]
    S[c[0],c[1],c[2],:] = i[0]
    M[c[0],c[1],c[2]] = i[1]
CSM=S*(M>(0.1*np.max(M)))[:,:,:,None]

In [69]:
#handy.plot_patterns(handy.swappy(abs(CSM.squeeze()[:,:,12])),4,'jet')

In [60]:
Nx,Ny,Nz,Nc

(96, 96, 20, 16)

In [61]:
a = np.arange(9).reshape((3,3))
a[0,1]

1

In [63]:
def permute(array,ind):
    mini = min(ind)
    nswaps = abs(ind[0]-ind[1])
    hulp = np.copy(array)
    for i in range(nswaps):
        print(mini+i,mini+i+1)
        hulp = np.swapaxes(hulp,mini+i,mini+i+1)
    return hulp

In [ ]:
dis = permute(img,[1,3])
dis.shape

In [ ]:
plt.imshow(abs(hulp[0,:,0,:]))